<a href="https://colab.research.google.com/github/Hari-MathWorks/TFLiteExploration/blob/main/LSTMinput_SequenceMode_layer_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(input_shape=(None,3),units= 100, return_sequences = True))

# Add a Dense layer with 10 units.
model.add(layers.Dense(5))
model.add(layers.Softmax())

model.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, None, 100)         41600     
_________________________________________________________________
dense_2 (Dense)              (None, None, 5)           505       
_________________________________________________________________
softmax_1 (Softmax)          (None, None, 5)           0         
Total params: 42,105
Trainable params: 42,105
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Step 2: Train & Evaluate the model.

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']);
x_test = np.zeros([3000,4,3], dtype=np.float32);
x_train = np.zeros([3000,4,3], dtype=np.float32);
y_test =  np.zeros([3000, 4, ], dtype=np.float32);
y_train = np.zeros([3000, 4, ], dtype=np.float32);

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
94/94 [==============================] - 2s 4ms/step - loss: 0.8044 - accuracy: 1.0000
Epoch 2/5
94/94 [==============================] - 0s 4ms/step - loss: 0.2107 - accuracy: 1.0000
Epoch 3/5
94/94 [==============================] - 0s 4ms/step - loss: 0.1179 - accuracy: 1.0000
Epoch 4/5
94/94 [==============================] - 0s 4ms/step - loss: 0.0604 - accuracy: 1.0000
Epoch 5/5
94/94 [==============================] - 1s 2ms/step - loss: 0.0223 - accuracy: 1.0000


[0.02233084850013256, 1.0]

In [8]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
BATCH_SIZE = 1
STEPS = 1000 #Change sequence length every time
INPUT_SIZE = 3

concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))


#concrete_func = run_model.get_concrete_function(
#    tf.TensorSpec([None, None, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "keras_lstm_new"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

with open('lstmmodelnew.tflite', 'wb') as f:
  f.write(tflite_model) 

INFO:tensorflow:Assets written to: keras_lstm_new/assets


INFO:tensorflow:Assets written to: keras_lstm_new/assets


In [15]:

interpreter = tf.lite.Interpreter(model_content=tflite_model)
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_path="lstmmodelnew.tflite")
#interpreter = tf.lite.Interpreter(model_path="lstmmodel_sequence_1_2_20.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()



In [16]:
print(input_details)
print(output_details)


[{'name': 'serving_default_x:0', 'index': 0, 'shape': array([   1, 1000,    3], dtype=int32), 'shape_signature': array([   1, 1000,    3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 19, 'shape': array([   1, 1000,    5], dtype=int32), 'shape_signature': array([   1, 1000,    5], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [17]:

input_shape = input_details[0]['shape']

output_shape = output_details[0]['shape']
print(input_shape) 
print(output_shape) 

[   1 1000    3]
[   1 1000    5]


In [18]:
myinp = np.loadtxt('Cmaj.txt')

OSError: ignored

In [ ]:
myinp = np.loadtxt('img11.txt')

In [ ]:
myinp = np.resize(myinp,input_shape);
myinp = (np.float32(myinp) - 0) / 1
print(myinp)

[[[0.085516 0.801015 0.928854 0.488609 0.237284 0.963089 0.521136
   0.488898 0.679136 0.367437 0.037739 0.913287 0.098712 0.335357
   0.136553 0.106762 0.494174 0.715037 0.890923 0.698746]
  [0.262482 0.02922  0.730331 0.578525 0.458849 0.546806 0.231594
   0.62406  0.395515 0.987982 0.885168 0.796184 0.261871 0.679728
   0.721227 0.653757 0.779052 0.903721 0.334163 0.19781 ]]]


In [ ]:
interpreter.allocate_tensors()
interpreter.reset_all_variables()
interpreter.set_tensor(input_details[0]["index"], myinp)
interpreter.invoke()
result = interpreter.get_tensor(output_details[0]["index"])
print(result)
print(result.shape)

[[[-0.0283664  -0.08022337  0.06505261  0.0116315  -0.0524184
    0.02021186 -0.04946107  0.01118309  0.0420268  -0.10148051]
  [ 0.00687851 -0.0870756   0.0696666   0.01911599 -0.07488528
    0.06445044 -0.04765568  0.004292    0.05890122 -0.10740543]]]
(1, 2, 10)
